# Creación de índice en Azure AI Search

To run this Notebook, you need to install Polyglot Notebooks for Visual Studio Code, which allows you to run .NET code in Jupyter Notebooks. You can find it here: [Polyglot Notebooks for Visual Studio Code](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode).

In [ ]:
// Instalar paquetes NuGet necesarios
#r "nuget: Azure.Search.Documents, *-*"
#r "nuget: Azure.Identity, *-*"
#r "nuget: DotNetEnv, 3.0.0"
#r "nuget: OpenAI, *-*"

### Definir un Azure AI Search índice

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Azure.Identity;
using Azure.Search.Documents.Indexes;
using Azure.Search.Documents.Indexes.Models;
using Azure.Search.Documents;
using DotNetEnv;
using OpenAI;
using OpenAI.Embeddings;
using System.ClientModel;


// Cargar variables de entorno
Env.Load();

string endpoint = Environment.GetEnvironmentVariable("AZURE_AI_SEARCH_ENDPOINT");
string githubToken = Environment.GetEnvironmentVariable("GITHUB_TOKEN");
var credential = new DefaultAzureCredential();

const string INDEX_NAME = "my-index-net";

In [ ]:
// Eliminar índice si existe
void DeleteIndex(SearchIndexClient client, string indexName)
{
    try
    {
        Console.WriteLine($"Eliminando índice {indexName}");
        client.DeleteIndex(indexName);
    }
    catch (Exception ex)
    {
        Console.WriteLine($"No se pudo eliminar el índice (puede no existir): {ex.Message}");
    }
}

In [ ]:
// Definir el índice con campo vectorial y configuración semántica
SearchIndex CreateIndexDefinition(string name)
{
    var fields = new List<SearchField>
    {
        new SimpleField("id", SearchFieldDataType.String) { IsKey = true },
        new SearchableField("title") {
                    IsFacetable = true,
                    IsFilterable = true,
                    AnalyzerName = LexicalAnalyzerName.EsMicrosoft,
                },
        new SearchableField("content") { AnalyzerName = LexicalAnalyzerName.EsMicrosoft },
        new SearchableField("keywords") {
                    IsFacetable = true,
                    IsFilterable = true,
                    AnalyzerName = LexicalAnalyzerName.EsMicrosoft,
                },
        new SearchField("embedding", SearchFieldDataType.Collection(SearchFieldDataType.Single))
        {
            IsSearchable = true,
            VectorSearchDimensions = 1536,
            VectorSearchProfileName = "myHnswProfile"
        }
    };

    var semanticConfig = new SemanticConfiguration("default",
        new() { ContentFields = { new SemanticField("content") },
                TitleField = new SemanticField("title"),
                KeywordsFields = { new SemanticField("keywords") },
            }
    );

    var vectorSearch = new VectorSearch()
    {
        Algorithms =  {
            new HnswAlgorithmConfiguration("myHnsw")
        },
        Profiles = {
            new VectorSearchProfile("myHnswProfile", "myHnsw")
        }
    };
    

    var semanticSearch = new SemanticSearch() { Configurations = {semanticConfig} };

    return new SearchIndex(name)
    {
        Fields = fields,
        SemanticSearch = semanticSearch,
        VectorSearch = vectorSearch
    };
}

## Crear el índice en Azure AI Search

In [ ]:
var indexClient = new SearchIndexClient(new Uri(endpoint), credential);

DeleteIndex(indexClient, INDEX_NAME);
Console.WriteLine($"Eliminado índice {INDEX_NAME}");
var index = CreateIndexDefinition(INDEX_NAME);
indexClient.CreateOrUpdateIndex(index);
Console.WriteLine($"Índice {INDEX_NAME} creado");

## Generar embeddings y preparar documentos

In [ ]:
const string MODEL_NAME = "openai/text-embedding-3-small";
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint = new Uri("https://models.github.ai/inference")
};
EmbeddingClient embeddingClient = new(MODEL_NAME, new ApiKeyCredential(githubToken), openAIOptions);

var contentInput = new[]
{
    new { Title = "Lionel Messi", Content = "Hoja de vida: Lionel Messi. Futbolista argentino, considerado uno de los mejores jugadores de fútbol de todos los tiempos." },
    new { Title = "Diego Zumárraga Mera", Content = "Hoja de Vida: Diego Zumárraga Mera. Ingeniero de software con experiencia en desarrollo de aplicaciones web y móviles, apasionado por la inteligencia artificial y el aprendizaje automático." }
};

var docs = new List<Dictionary<string, object>>();
for (int ix = 0; ix < contentInput.Length; ix++)
{
    var item = contentInput[ix];
    Console.WriteLine($"Procesando documento {ix}: {item.Content}");
    OpenAIEmbeddingCollection response = embeddingClient.GenerateEmbeddings(new List<string> { item.Content });
    var embedding = response.FirstOrDefault().ToFloats().ToArray();
    Console.WriteLine($"Embedding length: {embedding.Length}");

    docs.Add(new Dictionary<string, object>
    {
        ["id"] = $"doc-{ix}",
        ["title"] = item.Title,
        ["content"] = item.Content,
        ["keywords"] = item.Title.Replace(" ", ", "),
        ["embedding"] = embedding
    });
}
Console.WriteLine($"Generados {docs.Count} documentos para indexar.");

## Indexar documentos en Azure AI Search

In [ ]:
var searchClient = new SearchClient(new Uri(endpoint), INDEX_NAME, credential);
Console.WriteLine($"Indexando documentos...");
var uploadResult = searchClient.UploadDocuments(docs);
Console.WriteLine($"Subidos {docs.Count} documentos al índice {INDEX_NAME}");